In [17]:
import numpy as np
from sklearn import svm
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import joblib
import json

In [18]:
JSON_PATH = 'myData.json'

In [19]:
def load_data(json_path):
    with open(json_path, 'r') as dataset:
        data_str = dataset.read()
    data = json.loads(data_str)
    return data['mapping'], data['mfcc'], data['labels']

In [20]:
def test_model(model, X, y):
    kf = KFold(n_splits=10, shuffle=True)
    X = np.array(X[500:])
    y= np.array(y[500:])
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.25)
        train_model = model
        train_model.fit(X_train, y_train)
        y_validation_pred = train_model.predict(X_validation)
        y_train_pred = train_model.predict(X_train)
        y_test_pred = train_model.predict(X_test)
        mse_test = metrics.mean_squared_error(y_test, y_test_pred)
        mse_validation = metrics.mean_squared_error(y_validation, y_validation_pred)
        acc_test = metrics.accuracy_score(y_test, y_test_pred)
        acc_validation = metrics.accuracy_score(y_validation, y_validation_pred)
        precision_test = metrics.precision_score(y_test, y_test_pred, average='samples')
        precision_validation = metrics.precision_score(y_test, y_test_pred, average='samples')
        print(f'Accuracy on test: {acc_test}, Accuracy on validation: {acc_validation} \n'
              f'Precision on test: {precision_test}, Precision on validation: {precision_validation} \n'
              f'Loss on test: {mse_test}, Loss on validation: {mse_validation}\n')

    # folds = range(1, kf.get_n_splits() + 1)
    # plt.plot(folds, precision_test_lst, 'o-', color='green', label='train')
    # plt.plot(folds, precision_validation_lst, 'o-', color='red', label='test')
    # plt.legend()
    # plt.grid()
    # plt.xlabel('Number of fold')
    # plt.ylabel('Mean Squared Error')
    # plt.show()

In [21]:
_, inputs, targets = load_data(JSON_PATH)

In [22]:
def main(inputs, targets, kernel='linear'):
    model = svm.SVC(kernel=kernel, C=2)
    test_model(model, inputs, targets)

In [23]:
def save_model(model):
    svc = model
    joblib.dump(svc, 'last_svc.model')

In [24]:
main(inputs, targets)

ValueError: Samplewise metrics are not available outside of multilabel classification.